## Load Dependancies

In [1]:
import geopandas as gpd
from sqlalchemy import create_engine, text
from sqlalchemy_utils import create_database, database_exists, drop_database
import pandas as pd
import sqlalchemy

## Connect to Postgres DB

In [2]:
#Variables
db_type = "postgresql" #postgresql
username = "postgres"
password = "pass"
host = "localhost"
port = "5432"
db_name = "coviddistribution"

#Put it together
engine = create_engine(f"{db_type}://{username}:{password}@{host}:{port}/{db_name}")

In [3]:
inspector = sqlalchemy.inspect(engine)
inspector.get_table_names() #We see that our table was added to our database

['spatial_ref_sys',
 'uscounties',
 'vaccine_locations',
 'covid_cases',
 'vaccination_county',
 'demographics_county']

## Load Data

In [4]:
uscounties = gpd.read_file("data/UScounties/UScounties.shp")
airports = gpd.read_file("data/airports/airports.shp")
vaccine_locs = gpd.read_file("data/vaccine_locations/vaccine_locations.shp")

covid_cases = pd.read_csv("data/covid_cases_october.csv")
vacc_rate = pd.read_csv("data/vaccination_rate_by_county.csv")
minmaj = pd.read_csv("data/population_minmaj_county.csv")

### Minor Cleaning

In [5]:
airports.columns = ["icao","name","elevation_ft","state","state_fips","scheduled","keywords","is_military","geometry"]

covid_cases = covid_cases[covid_cases.columns[1:]]
vacc_rate = vacc_rate[vacc_rate.columns[1:]]
minmaj = minmaj[minmaj.columns[1:]]

## Write Tables to DB

In [6]:
tables_to_write = {
                   "uscounties":uscounties,
                   "airports":airports,
                   "vaccine_locations":vaccine_locs,
                   "covid_cases":covid_cases,
                   "vaccination_county":vacc_rate,
                   "demographics_county":minmaj
                  }

In [7]:
inspector = sqlalchemy.inspect(engine)
existing_tables = inspector.get_table_names()
for table_name,table in tables_to_write.items():
    if (table_name not in existing_tables):
        if (type(table)==gpd.GeoDataFrame):
            table.to_postgis(name=table_name, con=engine)
        else:
            table.to_sql(name=table_name,con=engine)

In [8]:
inspector = sqlalchemy.inspect(engine)
inspector.get_table_names() #We see that our table was added to our database

['spatial_ref_sys',
 'uscounties',
 'vaccine_locations',
 'airports',
 'covid_cases',
 'vaccination_county',
 'demographics_county']

## Test

In [9]:
#SQL query
sql = """
SELECT
    airports.icao,
    vaccine_locations."SiteID" as siteid,
    ST_Distance(airports.geometry,vaccine_locations.geometry) as dist,
    airports.geometry AS airport_geom,
    vaccine_locations.geometry as geometry
FROM 
    airports
INNER JOIN
    vaccine_locations ON
    airports.state_fips=vaccine_locations."STATE_FIPS"
WHERE 
    vaccine_locations."NAME"='Santa Clara' AND
    vaccine_locations."STATE_NAME"='California'
ORDER BY
    icao ASC,
    dist ASC
"""

#Specify name of column which stores our geometry! In table `streets_chapter_11`, the geometry is stored in a col called `geometry`
geom_col = "geometry"

#Execute query to create GeoDataFrame
ap = gpd.GeoDataFrame.from_postgis(sql=sql, con=engine, geom_col=geom_col)

In [10]:
ap

,icao,siteid,dist,airport_geom,geometry
0,KBAB,5372,1.764765,0101000020E6100000BD0A00C0F75B5EC061E8FFBF6B91...,POINT (-121.90178 37.43364)
1,KBAB,5372,1.764765,0101000020E6100000BD0A00C0F75B5EC061E8FFBF6B91...,POINT (-121.90178 37.43364)
2,KBAB,5364,1.796271,0101000020E6100000BD0A00C0F75B5EC061E8FFBF6B91...,POINT (-121.94675 37.41368)
3,KBAB,5364,1.796271,0101000020E6100000BD0A00C0F75B5EC061E8FFBF6B91...,POINT (-121.94675 37.41368)
4,KBAB,5367,1.796724,0101000020E6100000BD0A00C0F75B5EC061E8FFBF6B91...,POINT (-121.95437 37.41548)
...,...,...,...,...,...
1183,KVBG,5358,3.073221,0101000020E6100000FB39004060255EC07FFEFFDF5F5E...,POINT (-122.11441 37.40236)
1184,KVBG,5373,3.107657,0101000020E6100000FB39004060255EC07FFEFFDF5F5E...,POINT (-122.12905 37.43366)
1185,KVBG,5373,3.107657,0101000020E6100000FB39004060255EC07FFEFFDF5F5E...,POINT (-122.12905 37.43366)
1186,KVBG,5375,3.127454,0101000020E6100000FB39004060255EC07FFEFFDF5F5E...,POINT (-122.15864 37.43942)
